## Preprocessing

In [1]:
from helper import get_train_test, lowercase, tokenize, remove_punctuation, remove_stopwords, \
    remove_non_letters, stemming

hyperparameters = {
    'lowercase': True,
    'stemming': True,
    'remove_stopwords': True,
    'remove_non_letters': True,
    'remove_punctuation': True
}

train, test = get_train_test('train_data.txt', 'test_data.txt')

# lowercasing
if hyperparameters['lowercase']:
    lowercase(train)
    lowercase(test)

# removing non letters
if hyperparameters['remove_non_letters']:
    remove_non_letters(train)
    remove_non_letters(test)

# removing punctuation
if hyperparameters['remove_punctuation']:
    remove_punctuation(train)
    remove_punctuation(test)

# tokenizing
tokenize(train)
tokenize(test)

# removing stopwords
if hyperparameters['remove_stopwords']:
    remove_stopwords(train)
    remove_stopwords(test)
    
# stemming
if hyperparameters['stemming']:
    stemming(train)
    stemming(test)

## Classification

In [2]:
# TODO
train['subject']

[['anyon', 'right', 'send', 'song', 'plz', 'plz', 'plz', 'plz', 'plz'],
 ['ford', 'probe', 'flat', 'spot', 'rpm', 'rpm', 'common'],
 ['wont', 'volvo', 'start'],
 ['seroiu', 'relationship', 'prob'],
 ['im', 'look', 'song'],
 ['find', 'format', 'quitclaim', 'regard', 'employ'],
 ['ladi', 'definit', 'sexi'],
 ['nail'],
 ['would', 'possibl', 'u', 'buy', 'cananda'],
 ['wear', 'spa'],
 ['affirm', 'side', 'legal', 'prostitut', 'philippin'],
 ['urin', 'suppos', 'happen'],
 ['market', 'valu', 'veget', 'protein'],
 ['tallest', 'tower', 'world', 'mani', 'stair'],
 ['calcul', 'someth', 'occur', 'instant'],
 ['els', 'think', 'melina', 'reveal', 'night', 'battl', 'royal'],
 ['mazda', 'mpv', 'spark'],
 ['good', 'excus', 'drink', 'alcohol'],
 ['sure', 'work'],
 ['reason',
  'woman',
  'get',
  'pregnant',
  'would',
  'woman',
  'know',
  'she',
  'steral'],
 ['inde', 'learn', 'someth', 'new'],
 ['hey', 'anybodi', 'know', 'one', 'realli', 'scari', 'movi', 'play', 'tv'],
 ['could',
  'proof',
  'tan',
